In [3]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar

try: import setGPU
except ImportError: pass
import warnings
import torch

import model_classes, nets
from constants import *

from model_classes import SolvePointQP

from data import get_data, get_decision_mask, get_decision_mask_

%load_ext autoreload
%autoreload 2

warnings.simplefilter("ignore")

DEVICE = "cpu"
if torch.cuda.is_available(): 
    DEVICE = "cuda"
print("DEVICE", DEVICE, "hi")

X_train, Y_train, X_test, Y_test, X_train_pt, Y_train_pt, X_test_pt, Y_test_pt = get_data(DEVICE) 
variables = {'X_train_': X_train_pt, 'Y_train_': Y_train_pt, 
        'X_test_': X_test_pt, 'Y_test_': Y_test_pt}

params = {"n": 24, "c_ramp": 0.4, "gamma_under": 50, "gamma_over": 0.5}

mask = get_decision_mask(Y_train_pt, DEVICE)

EPOCHS_rmse = 200

print("TRAINING 2-STAGE")
model_rmse = model_classes.Net(X_train[:,:-1], Y_train, [200, 200]).to(DEVICE)
model_rmse = nets.run_rmse_net(model_rmse, variables, X_train, Y_train, EPOCHS_rmse)
model_rmse.eval()

# p = model_rmse(X_test_pt)[0]
# solver = SolvePointQP(params, DEVICE=DEVICE)
# decision = solver(p)[:,:params["n"]]

EPOCHS_e2e = 200

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DEVICE cuda hi
TRAINING 2-STAGE


In [4]:
var = 0.0

In [118]:
print("TRAINING END-to-END")
end_to_end_net = nets.train_end_to_end(X_train[:,:-1], Y_train, variables, params, 400, DEVICE)
end_to_end_net.eval()

TRAINING END-to-END
epoch: 0   6.613633608818054
epoch: 40   0.5205542981624603
epoch: 80   0.5315967857837677
epoch: 120   0.43489197492599485
epoch: 160   0.4628221869468689
epoch: 200   0.4122711718082428
epoch: 240   0.3972201973199844
epoch: 280   0.4096223175525665
epoch: 320   0.40979216396808626
epoch: 360   0.44858658611774443


Net(
  (lin): Linear(in_features=149, out_features=24, bias=True)
  (net): Sequential(
    (0): Linear(in_features=149, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=24, bias=True)
  )
)

In [156]:
print("TRAIN P-MODEL")
p_net = nets.train_pnet(end_to_end_net, X_train_pt, Y_train_pt, params, 1000, DEVICE, var)
p_net.eval();

TRAIN P-MODEL
epoch: 0   0.6230655077558297
epoch: 100   0.3669359654188156
epoch: 200   0.35236798018217086
epoch: 300   0.34496688410639764
epoch: 400   0.3383945734798908
epoch: 500   0.33079259902238844
epoch: 600   0.3238826900720596
epoch: 700   0.3286334276199341
epoch: 800   0.30682149410247805
epoch: 900   0.29618704795837403


In [117]:
# for ww in range(24):
#     m = get_decision_mask_(Y_train_pt[:1,:], DEVICE, ww)
#     print(m)
#     print(p_net(X_train_pt[:1,:], Y_train_pt[:1,:], m, 0))

In [104]:
p_net(X_train_pt[:1,:], Y_train_pt[:1,:], , 0)

tensor([[1.6723, 1.5864, 1.5555, 1.5367, 1.5365, 1.5746, 1.6495, 1.8035, 1.8671,
         1.8977, 1.9413, 1.9496, 1.9649, 1.9703, 1.9551, 1.9433, 1.9385, 1.9746,
         2.0890, 2.1350, 2.1326, 2.1020, 2.0524, 1.8878]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [ ]:
print("TRAIN F-MODEL")
# f_net = model_classes.FNet(X_train[:,:-1], Y_train, [200, 200]).to(DEVICE)
f_net = nets.train_fnet(f_net, p_net, X_train[:,:-1], Y_train, variables, params, 1000, DEVICE, var=var)

end_to_end_net.eval()
f_net.eval()

TRAIN F-MODEL
epoch: 0   0.8602244783945096
epoch: 10   0.20670923786703496


In [211]:
def task_loss_no_mean(Y_sched, Y_actual, params):
    return (params["gamma_under"] * torch.clamp(Y_actual - Y_sched, min=0) + 
        params["gamma_over"] * torch.clamp(Y_sched - Y_actual, min=0))

def pred_obj(x, w): 
    f_pred = f_net(x)[:, w*24 : w*24 + 24]
    m = get_decision_mask_(f_pred, DEVICE, w)

    p_pred = p_net(x, f_pred, m, var)

    v2 = p_pred[:,w:]
    v1 = p_net(x, torch.zeros_like(f_pred), torch.zeros_like(f_pred), var)[:,:w]
    # v1 = f_pred[:,:w]
    
    # print(e_pred)
    v = torch.cat((v1, v2), 1)

    loss = nets.task_loss(v, f_pred, params).mean()
    return loss.item()


def eval_decision(x, y, w):
    f_pred = f_net(x)[:, w*24 : w*24 + 24]
    m = get_decision_mask_(y, DEVICE, w)

    p_pred = p_net(x, y, m, var)
    
    v1 = p_net(x, y, torch.zeros_like(y), var)[:,:w]
    # v1 = f_pred[:,:w]
    v2 = p_pred[:,w:]
            
    v = torch.cat((v1, v2), 1)

    loss = nets.task_loss(v, y, params).mean()
    return loss.item()

# def task_loss_no_mean(Y_sched, Y_actual, params):
#     return (params["gamma_under"] * torch.clamp(Y_actual - Y_sched, min=0) + 
#         params["gamma_over"] * torch.clamp(Y_sched - Y_actual, min=0))

# def pred_obj(x, w): 
#     f_pred = f_net(x)[:, w*24 : w*24 + 24]
#     m = get_decision_mask_from_w(f_pred, DEVICE, w)

#     p_pred = p_net(x, f_pred, m, var)

#     v2 = p_pred[:,w:]
#     v1 = p_net(x, torch.zeros_like(f_pred), torch.zeros_like(f_pred), var)[:,:w]
#     # v1 = f_pred[:,:w]
    
#     # print(e_pred)
#     v = torch.cat((v1, v2), 1)

#     loss = nets.task_loss_no_mean(v, f_pred, params)
#     return loss


# def eval_decision(x, y, w):
#     f_pred = f_net(x)[:, w*24 : w*24 + 24]
#     m = get_decision_mask_from_w(y, DEVICE, w)

#     p_pred = p_net(x, y, m, var)
    
#     v1 = p_net(x, y, torch.zeros_like(y), var)[:,:w]
#     # v1 = f_pred[:,:w]
#     v2 = p_pred[:,w:]
            
#     v = torch.cat((v1, v2), 1)

#     loss = nets.task_loss_no_mean(v, y, params)
#     return loss


In [150]:
end_to_end_net(X_t)

tensor([[2.1740, 2.0273, 1.8970, 1.8477, 1.8164, 1.7363, 1.8642, 1.9320, 2.0858,
         2.2022, 2.3799, 2.4171, 2.6109, 2.7183, 2.7973, 2.8088, 2.9017, 2.9524,
         2.8470, 2.7919, 2.6478, 2.5020, 2.5507, 2.3695]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [151]:
p_net(X_t, Y_t, torch.zeros_like(Y_t), var)

tensor([[2.0128, 1.9160, 1.7342, 1.7035, 1.6818, 1.6117, 1.7197, 1.8277, 1.9460,
         2.1450, 2.2839, 2.2237, 2.4866, 2.5499, 2.6528, 2.5992, 2.7665, 2.8973,
         2.7627, 2.6423, 2.5201, 2.3392, 2.3922, 2.2007]], device='cuda:0',
       grad_fn=<AddBackward0>)

In [188]:
vanilla = [] 
ours = [] 
random = []
optimal = []

for indx in range(len(X_test_pt)):
# for indx in [2]:
    X_t = X_test_pt[indx:indx+1,:]
    Y_t = Y_test_pt[indx:indx+1,:]
    # X_t = X_train_pt[indx:indx+1,:]
    # Y_t = Y_train_pt[indx:indx+1,:]

    # e2e = nets.task_loss(p_net(X_t, Y_t, torch.zeros_like(Y_t), var), Y_t, params).mean().item()
    e2e_pred = p_net(X_t, torch.zeros_like(Y_t), torch.zeros_like(Y_t), var)
    e2e = nets.task_loss(e2e_pred, Y_t, params).mean().item() 
    ts = nets.task_loss(model_rmse.predict(X_t), Y_t, params).mean().item()

    print("TRUE")
    best_cost = 1e5 
    best_dec = -1
    for w in range(24): 
        cur = eval_decision(end_to_end_net, X_t, Y_t, w) + a*w
        print(w, cur)
        if cur < best_cost: 
            best_cost = cur 
            best_dec = w
            
    optimal.append(best_cost)
    vanilla.append(e2e)    

    print("end-to-end cost:", e2e)
    print("2-stage    cost:", ts)
    
    print("OPTIMAL true cost:", best_dec, best_cost)

TRUE
0 0.23849332332611084
1 0.27627032995224
2 0.28787314891815186
3 0.2841285467147827
4 0.2756218910217285
5 0.2786579132080078
6 0.26621580123901367
7 0.2676845192909241
8 0.2602653205394745
9 0.2594180703163147
10 0.259797066450119
11 0.265658438205719
12 0.25898313522338867
13 0.2514016628265381
14 0.24899153411388397
15 0.24317708611488342
16 0.23874607682228088
17 0.23659536242485046
18 0.23383407294750214
19 0.23710867762565613
20 0.23156476020812988
21 0.23592494428157806
22 0.23588547110557556
23 0.2368556708097458
end-to-end cost: 0.23849332332611084
2-stage    cost: 0.38677239418029785
OPTIMAL true cost: 20 0.23156476020812988
TRUE
0 0.20383362472057343
1 0.2488512247800827
2 0.2603912353515625
3 0.255912184715271
4 0.24763962626457214
5 0.25172945857048035
6 0.23874354362487793
7 0.23950359225273132
8 0.23164823651313782
9 0.2298741638660431
10 0.23028765618801117
11 0.23547053337097168
12 0.2276872843503952
13 0.22035539150238037
14 0.21814855933189392
15 0.2119812965393

KeyboardInterrupt: 

In [164]:
print("VANILLA e2e", np.mean(vanilla), np.std(vanilla))
print("2-stage    ", np.mean(ts), np.std(ts))
print("optimal    ", np.mean(optimal), np.std(optimal))

VANILLA e2e 0.5083838138315413 0.8831501980562387
2-stage     1.4612315893173218 0.0
optimal     0.3005937583837804 0.18822885611097778


In [270]:
vanilla = [] 
ours = [] 
random = []
optimal = []
two_stage = []
all_costs = []
end_to_end_net.eval()
f_net.eval()

var = 0.0

for indx in range(len(X_test_pt)):
# for indx in [2]:
    X_t = X_test_pt[indx:indx+1,:]
    Y_t = Y_test_pt[indx:indx+1,:]
    # X_t = X_train_pt[indx:indx+1,:]
    # Y_t = Y_train_pt[indx:indx+1,:]

    # e2e = nets.task_loss(end_to_end_net.predict(X_t), Y_t, params).mean().item()
    e2e_pred = p_net(X_t, torch.zeros_like(Y_t), torch.zeros_like(Y_t), var)
    e2e = nets.task_loss(e2e_pred, Y_t, params).mean().item() 

    ts = nets.task_loss(model_rmse.predict(X_t), Y_t, params).mean().item()
    
    print("end-to-end cost:", e2e)
    print("2-stage    cost:", ts)

    a = 0.0
    best_cost = 1e5 
    best_w = -1
    cur_costs = [] 
    for w in range(24):
        c = pred_obj(X_t, w)
        # print(w, c)
        if c < best_cost: 
            best_cost = c
            best_w = w
            
    our = eval_decision(X_t, Y_t, best_w) + a*best_w

    r = np.random.randint(24)
    rand = eval_decision(X_t, Y_t, r) + a*r
    
    print("TRUE")
    best_cost = 1e5 
    best_dec = -1
    uniform_random = 0
    for w in range(24): 
        cur = eval_decision(X_t, Y_t, w) + a*w
        # print(w, cur)
        if cur < best_cost: 
            best_cost = cur 
            best_dec = w
        uniform_random += cur
        cur_costs.append(cur)
    all_costs.append(cur_costs)
            
    # if best_dec == 0: continue 
    
    optimal.append(best_cost)
    vanilla.append(e2e)    
    ours.append(our)
    random.append(uniform_random/24)
    two_stage.append(ts)
    
    print("CHOSEN  true cost:", best_w, our)
    print("RANDOM  true cost: ", -1, uniform_random/24)
    print("OPTIMAL true cost:", best_dec, best_cost)
    print()


end-to-end cost: 0.23849332332611084
2-stage    cost: 0.38677239418029785
TRUE
CHOSEN  true cost: 8 0.2602653205394745
RANDOM  true cost:  -1 0.254548034320275
OPTIMAL true cost: 20 0.23156476020812988

end-to-end cost: 0.20383362472057343
2-stage    cost: 0.27114337682724
TRUE
CHOSEN  true cost: 8 0.23164823651313782
RANDOM  true cost:  -1 0.22379719155530134
OPTIMAL true cost: 20 0.19790172576904297

end-to-end cost: 0.24790582060813904
2-stage    cost: 0.3690739572048187
TRUE
CHOSEN  true cost: 15 0.25542712211608887
RANDOM  true cost:  -1 0.2671785832693179
OPTIMAL true cost: 20 0.2410213202238083

end-to-end cost: 0.24303756654262543
2-stage    cost: 0.2576451003551483
TRUE
CHOSEN  true cost: 8 0.26226896047592163
RANDOM  true cost:  -1 0.25674219739933807
OPTIMAL true cost: 20 0.23442670702934265

end-to-end cost: 0.2750511169433594
2-stage    cost: 1.5390875339508057
TRUE
CHOSEN  true cost: 8 0.2954566776752472
RANDOM  true cost:  -1 0.2917058865229289
OPTIMAL true cost: 20 0.27

In [224]:
from data import get_decision_mask_from_w

all_pred = []
for w in range(24):
    c = pred_obj(X_test_pt, torch.ones(len(X_test_pt)).long() * w)
    all_pred.append(list(c.cpu().detach().numpy()))
all_pred = torch.tensor(all_pred)
action_chosen = torch.argmin(all_pred, dim=0)
mask = get_decision_mask_from_w(X_test_pt, action_chosen, DEVICE)

my_decisions = eval_decision(X_t, Y_t, action_chosen)

0.44968341588974003


In [271]:
print("VANILLA e2e", np.mean(vanilla), np.std(vanilla))
print("Our app   ", np.mean(ours), np.std(ours))
print("random", np.mean(random), np.std(random))
print("optimal    ", np.mean(optimal), np.std(optimal))
avg_cost_per_action = np.mean(all_costs, axis=0)
# print(np.min(avg_cost_per_action))
print(avg_cost_per_action[8])

VANILLA e2e 0.5083838138315413 0.8831501980562387
Our app    0.38059068443209915 0.3967123031388382
random 0.40999309175215903 0.4715996221943384
optimal     0.3005937583837804 0.18822885611097778
0.37982527020858114


In [259]:
print("VANILLA e2e", np.median(vanilla), np.std(vanilla))
print("Our app   ", np.median(ours), np.std(ours))
print("random", np.median(random), np.std(random))
print("optimal    ", np.median(optimal), np.std(optimal))

VANILLA e2e 0.32994768023490906 0.8831501980562387
Our app    0.32519617676734924 0.41531367774910216
random 0.3367307757337888 0.4715996221943384
optimal     0.301961213350296 0.18822885611097778


In [260]:
QUANTILE = 0.5
print("VANILLA e2e", np.quantile(vanilla, QUANTILE), np.std(vanilla))
print("Our app   ", np.quantile(ours, QUANTILE), np.std(ours))
print("random", np.quantile(random, QUANTILE), np.std(random))
print("optimal    ", np.quantile(optimal, QUANTILE), np.std(optimal))

# best single action 
avg_cost_per_action = np.quantile(all_costs, QUANTILE, axis=0)
print(avg_cost_per_action[20])

VANILLA e2e 0.32994768023490906 0.8831501980562387
Our app    0.32519617676734924 0.41531367774910216
random 0.3367307757337888 0.4715996221943384
optimal     0.301961213350296 0.18822885611097778
0.32208365201950073


In [235]:
print(sum(np.array(ours) < np.array(random)) / len(ours))

0.8685446009389671


In [255]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(24)], y=, mode='lines', name='data', marker=dict(color='green')))
fig.update_layout(xaxis_title='alpha', yaxis_title='Loss', title='Loss at w = 0.25, g(w,z)=1', width = 600, height = 600)
